# 🍓 딸기 OK/NG 분류 모델 학습 (Google Colab)

YOLO11 기반 딸기 품질 분류 모델을 학습합니다.

**런타임 설정**: 런타임 > 런타임 유형 변경 > **T4 GPU** 선택

---


## 1️⃣ 환경 설정


In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# 필요한 패키지 설치
%pip install -q ultralytics
%pip install -q sahi
%pip install -q onnxruntime

print("✅ 패키지 설치 완료!")


In [ ]:
# GPU 확인
import torch
print(f"🔥 CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"📊 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")


## 2️⃣ 프로젝트 설정


In [ ]:
# GitHub에서 코드 클론
import os
from pathlib import Path

# 작업 디렉토리 설정
WORK_DIR = "/content/drive/MyDrive/ComputerVision"

# 디렉토리가 없으면 생성 및 클론
if not os.path.exists(WORK_DIR):
    print("📥 GitHub에서 프로젝트 클론...")
    !git clone https://github.com/adjective123/ComputerVision.git /content/drive/MyDrive/ComputerVision
else:
    print("✅ 프로젝트 디렉토리 존재")
    # 최신 코드 pull
    %cd {WORK_DIR}
    !git pull origin main

%cd {WORK_DIR}
print(f"\n📁 현재 작업 디렉토리: {os.getcwd()}")


In [ ]:
# config.py와 YAML 파일의 경로를 Colab 환경에 맞게 수정
import re

# config.py 업데이트
config_path = f"{WORK_DIR}/config.py"
with open(config_path, 'r', encoding='utf-8') as f:
    config_content = f.read()

# BASE_DIR 경로 변경
config_content = re.sub(
    r'BASE_DIR = Path\("/Users/temp/내 드라이브\(codejeteho123@gmail\.com\)/ComputerVision"\)',
    '# BASE_DIR = Path("/Users/temp/내 드라이브(codejeteho123@gmail.com)/ComputerVision")',
    config_content
)
config_content = re.sub(
    r'# BASE_DIR = Path\("/content/drive/MyDrive/ComputerVision"\)',
    'BASE_DIR = Path("/content/drive/MyDrive/ComputerVision")',
    config_content
)

with open(config_path, 'w', encoding='utf-8') as f:
    f.write(config_content)

# YAML 파일 업데이트
yaml_path = f"{WORK_DIR}/strawberry_ok_ng.yaml"
with open(yaml_path, 'r', encoding='utf-8') as f:
    yaml_content = f.read()

yaml_content = re.sub(
    r'path: /Users/temp/내 드라이브\(codejeteho123@gmail\.com\)/ComputerVision/딸기_binary_dataset',
    'path: /content/drive/MyDrive/ComputerVision/딸기_binary_dataset',
    yaml_content
)

with open(yaml_path, 'w', encoding='utf-8') as f:
    f.write(yaml_content)

print("✅ 경로 업데이트 완료!")


## 3️⃣ 데이터셋 생성 및 확인

이 단계에서 JSON 라벨 파일을 YOLO 포맷으로 변환하고 train/val로 분할합니다.

**필요한 데이터:**
- `딸기 라벨링 JSON/` 폴더: JSON 라벨 파일
- `딸기이미지/` 폴더: 원본 이미지

**생성되는 파일:**
- `딸기_binary_dataset/`: YOLO 포맷 데이터셋 (images, labels)
- `strawberry_ok_ng.yaml`: 데이터셋 설정 파일


In [ ]:
# 데이터셋 구조 확인
data_dir = Path(WORK_DIR) / "딸기_binary_dataset"
yaml_file = Path(WORK_DIR) / "strawberry_ok_ng.yaml"

# 데이터셋이 없으면 생성
if not data_dir.exists() or not yaml_file.exists():
    print("📦 데이터셋 생성 중...\n")
    print("=" * 70)
    
    # dataset_preparation.py 실행
    !python dataset_preparation.py
    
    print("\n" + "=" * 70)
    print("✅ 데이터셋 생성 완료!\n")
else:
    print("✅ 데이터셋이 이미 존재합니다.\n")

# 데이터셋 구조 출력
if data_dir.exists():
    print("📊 데이터셋 구조:")
    print("=" * 70)
    
    total_images = 0
    total_labels = 0
    
    for split in ['train', 'val']:
        img_dir = data_dir / 'images' / split
        label_dir = data_dir / 'labels' / split
        
        if img_dir.exists():
            num_images = len(list(img_dir.glob('*.jpg')))
            num_labels = len(list(label_dir.glob('*.txt')))
            total_images += num_images
            total_labels += num_labels
            print(f"  {split.upper():5s}: 이미지 {num_images:4d}개, 라벨 {num_labels:4d}개")
    
    print("=" * 70)
    print(f"  총합 : 이미지 {total_images:4d}개, 라벨 {total_labels:4d}개")
    print("=" * 70)
else:
    print("❌ 데이터셋 생성에 실패했습니다.")


## 4️⃣ 모델 학습 🚀

**예상 소요 시간**: T4 GPU 기준 약 1-2시간


In [ ]:
# YOLO 모델 학습
from ultralytics import YOLO

print("🚀 YOLO 모델 학습 시작...\n")

# 모델 로드 (yolo11s.pt 사용)
model = YOLO('yolo11s.pt')

# 학습 실행
results = model.train(
    data='strawberry_ok_ng.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    patience=20,
    workers=2,
    device=0,  # GPU 사용
    name='strawberry_ok_ng',
    project='runs/detect',
    
    # 최적화 설정
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    momentum=0.9,
    weight_decay=0.0005,
    warmup_epochs=3,
    warmup_momentum=0.8,
    box=7.5,
    cls=0.5,
    dfl=1.5,
    
    # 데이터 증강 (색상 변화에 강건)
    hsv_h=0.03,        # Hue (색상): ±3%
    hsv_s=0.8,         # Saturation (채도): ±80%
    hsv_v=0.5,         # Value (명도): ±50%
    degrees=15.0,      # 회전: ±15도
    translate=0.15,    # 이동: ±15%
    scale=0.6,         # 크기: ±60%
    shear=2.0,         # 전단: ±2도
    perspective=0.0001,  # 원근
    flipud=0.0,        # 상하 반전 (딸기는 방향성)
    fliplr=0.5,        # 좌우 반전: 50%
    mosaic=1.0,        # Mosaic 증강
    mixup=0.15,        # MixUp 증강: 15%
    copy_paste=0.1     # Copy-Paste 증강: 10%
)

print("\n" + "="*70)
print("✅ 학습 완료!")
print("="*70)


## 5️⃣ 학습 결과 확인 📊


In [ ]:
# 학습 결과 시각화
from IPython.display import Image, display

results_dir = Path(WORK_DIR) / 'runs' / 'detect' / 'strawberry_ok_ng'

# 주요 결과 이미지 표시
result_images = [
    ('results.png', '📈 학습 메트릭'),
    ('confusion_matrix.png', '🔍 Confusion Matrix'),
    ('F1_curve.png', '📊 F1 Score'),
    ('PR_curve.png', '📊 Precision-Recall'),
]

for img_name, title in result_images:
    img_path = results_dir / img_name
    if img_path.exists():
        print(f"\n{title}")
        print("="*70)
        display(Image(filename=str(img_path), width=800))
    else:
        print(f"⚠️ {img_name} 없음")


In [ ]:
# 모델 평가
best_model = YOLO(str(results_dir / 'weights' / 'best.pt'))

print("📊 모델 평가 중...\n")
metrics = best_model.val(
    data='strawberry_ok_ng.yaml',
    split='val',
    batch=16,
    device=0
)

print("\n" + "="*70)
print("📊 최종 평가 결과")
print("="*70)
print(f"  mAP@50      : {metrics.box.map50*100:.2f}%")
print(f"  mAP@50-95   : {metrics.box.map*100:.2f}%")
print(f"  Precision   : {metrics.box.mp*100:.2f}%")
print(f"  Recall      : {metrics.box.mr*100:.2f}%")
print("="*70)


## 6️⃣ 모델 내보내기 (ONNX)


In [ ]:
# ONNX 형식으로 변환
print("🔄 ONNX 형식으로 변환 중...\n")

# ONNX로 변환
onnx_path = best_model.export(format='onnx')

print(f"\n✅ ONNX 변환 완료!")
print(f"📁 저장 위치: {onnx_path}")
print(f"\n💡 Google Drive에 저장되어 있으므로 언제든지 접근 가능합니다!")


## 7️⃣ 요약 및 다음 단계

### ✅ 완료된 작업
1. Google Drive 마운트
2. 프로젝트 코드 클론/업데이트
3. 데이터셋 확인
4. YOLO 모델 학습 (강화된 데이터 증강 포함)
5. 학습 결과 확인
6. 모델 평가
7. 모델 내보내기 (ONNX)

### 📁 저장된 파일
- **모델**: `/content/drive/MyDrive/ComputerVision/runs/detect/strawberry_ok_ng/weights/`
  - `best.pt`: 최고 성능 PyTorch 모델
  - `best.onnx`: ONNX 변환 모델
  - `last.pt`: 마지막 에포크 모델

### 🚀 다음 단계
1. **로컬에서 테스트**: 모델을 사용하여 `test_sahi.py`로 테스트
```bash
python test_sahi.py --image sample.jpg --conf 0.85
```

2. **배치 테스트**: `batch_test_images.py`로 여러 이미지 테스트
```bash
python batch_test_images.py --num-samples 20 --conf 0.85
```

3. **성능 개선**: 잘못 분류된 이미지를 분석하여 데이터 추가

---

**💡 Tip**: 이 노트북을 Google Drive에 저장하면 다음에 다시 사용할 수 있습니다!

**🎯 GitHub**: https://github.com/adjective123/ComputerVision
